In [1]:
import os, code, sys, time
import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L 
import chainer.optimizers as optimizers
import chainer.serializers as serializers
from chainer import cuda
import cupy
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
from data_utils import *
import models, nn, graph_ops
import gc

In [9]:
thetas = np.load('thetas_timesteps.npy').item()
rs = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.5, 2.0, 4.0, 6.0]
rs = rs[::-1]
#rs_tups = {rs[i]:[rs[j] for j in range(i+1,len(rs))] for i in range(len(rs)-1)}
rs_tups = [(rs[i], rs[j]) for i in range(len(rs)) for j in range(i+1, len(rs))]
lh_dir = './Close/'
set_name = lh_dir + 'set_16_{}_{}_LossHistory.npy'
graph_name = lh_dir + 'kgraph_16_{}_{}_LossHistory.npy'
greatest_diff = {'diff':0.0, 'rs':(6.0,4.0), 'lh':0.4324}

In [23]:
lhmeans = []
lhdiffs = []
lhrs = []
for zX, zY in rs_tups:
    for use_graph in [False, True]:
        cur_theta = thetas[(16,zX,zY)]['lim']
        model = set_name if not use_graph else graph_name
        cur_lh = np.load(model.format(zX,zY))
        cur_mean = np.mean(cur_lh[-50:])
        diff = abs(cur_theta - cur_mean)
        mtype = 'G' if use_graph else 'S'
        if cur_mean < cur_theta:
            #print('{}, {}'.format(zX,zY))
            if mtype != 'S':
                lhmeans.append(cur_mean)
                lhdiffs.append(diff)
                lhrs.append((zX,zY))
            #lhmeans.append(cur_mean)
            print('{} LT {} {}, lim: {}, lh: {}'.format(mtype, zX,zY, cur_theta, cur_mean))
            if diff > greatest_diff['diff']:
                greatest_diff['diff'] = diff
                greatest_diff['rs'] = (zX,zY)
                greatest_diff['lh'] = cur_mean
        #else:
            #print('{} gt {} {}, lim: {}, lh: {}'.format(mtype, zX,zY, cur_theta, cur_mean))
lhmeans = np.array(lhmeans)
lhdiffs = np.array(lhdiffs)
lhrs = np.array(lhrs)

G LT 6.0 0.4, lim: 0.002086424268782139, lh: 0.0020443028770387173
G LT 6.0 0.2, lim: 0.0037951532285660505, lh: 0.0036190086975693703
S LT 6.0 0.0, lim: 0.00660347705706954, lh: 0.0056159887462854385
G LT 6.0 0.0, lim: 0.00660347705706954, lh: 0.005365326534956694
G LT 4.0 0.0, lim: 0.004940531682223082, lh: 0.004743741825222969
G LT 2.0 1.2, lim: 5.881043762201443e-05, lh: 5.6276599934790283e-05
G LT 2.0 1.0, lim: 0.00013450681581161916, lh: 0.00013234041398391128
G LT 2.0 0.8, lim: 0.00028291717171669006, lh: 0.0002630873932503164
G LT 2.0 0.6, lim: 0.0005468276212923229, lh: 0.000498873763717711
G LT 2.0 0.4, lim: 0.0009804371511563659, lh: 0.0008628673967905343
G LT 2.0 0.2, lim: 0.0017030484741553664, lh: 0.001474065356887877
G LT 2.0 0.0, lim: 0.002969615161418915, lh: 0.002555545186623931
G LT 1.5 0.6, lim: 0.00023963983403518796, lh: 0.0002184123295592144
G LT 1.5 0.4, lim: 0.000537640880793333, lh: 0.0004538671928457916
G LT 1.5 0.2, lim: 0.0010522492229938507, lh: 0.00084903

In [ ]:
y = loss_history[:cur_iter+1]
plt.close('all')
title = 'n_P {} RS {}, Iteration: {}, loss: {:.4}'.format(nparticles, redshift_tuple, cur_iter+1, y[-1])
fig = plot_training_curve(y, cur_iter, yclip=0.01, title=title)
display.display(plt.gcf())
display.clear_output(wait=True)
time.sleep(0.0001)

In [ ]:
zX = 
X = normalize(load_datum(zX, nparticles), cupy_out=True)
Y = normalize(load_datum(zY, nparticles), cupy_out=True)